<a href="https://colab.research.google.com/github/chanakatb/pytorch_deep_learning/blob/main/Create_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Deep Learning with PyTorch 10**

**Convolutional Neural Network Intro**

*minist dats set*: *https://www.tensorflow.org/datasets/catalog/mnist*

Training data set includes 60k images and test data set includes 10k images.

## **Deep Learning with PyTorch 11**

**Image Filter / Image Kernel Overview**

*Website for image filters (image Kernels): https://setosa.io/ev/image-kernels/*

## **Deep Learning with PyTorch 12**

**Convolutional Layer and RGB**

In CNN, neurons are not fully connected and are locally connected.

A typical CNN: https://editor.analyticsvidhya.com/uploads/59954intro%20to%20CNN.JPG

## **Deep Learning with PyTorch 13**

**Pooling Layer in Convolutional Neural Network**